In [1]:
from google.colab import drive
import os

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/DataFusionContest/')

Mounted at /content/gdrive


In [2]:
# Все, что выше - удалить

In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score, accuracy_score
import io
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json

from tensorflow.keras import Input, Model, models, optimizers
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU, concatenate, Flatten, Embedding
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint

SLICE = 1000000

In [4]:
# Чтение датасета
df = pd.read_parquet('train.parquet', columns=['item_name', 'category_id'])
df = df.query('category_id >= 0')
df['item_name'] = df['item_name'].astype(str)
df['item_name'] = df['item_name'].str.lower()

In [5]:
# Чтение названий колонок результата работы модели
z = pd.read_csv('columns.csv')
dummiesColumnsName = list(z.columns)

In [6]:
# Загрузка словаря токенайзера
with open('tokenizer_item_name.json') as f:
    data = json.load(f)
    tokenizer_goods = tokenizer_from_json(data)

In [7]:
# Загрузка модели
#from tensorflow.keras import Input, Model, regularizers, optimizers, models

#load structure
json_file = open('model_v3.json', "r")
loaded_model_json = json_file.read()
json_file.close()

model2 = models.model_from_json(loaded_model_json)
# load weights
model2.load_weights('model_v3.h5')
print("Done")
model = Model(model2.input, model2.layers[-1].output)
#model.trainable = True

Done


In [8]:
loss='categorical_crossentropy'#binary_crossentropy categorical_crossentropy
optimiser=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
metrics1='accuracy'
model.compile(optimizer=optimiser, 
              loss=loss, 
              metrics=[metrics1])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 64)            1280000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 64)            33024     
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               98816     
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 98)                12642 

In [9]:
# Обработка названий товаров по полученному словарю
sequences_goods = tokenizer_goods.texts_to_sequences(df['item_name'])
max_position_len = 10
# Замена неожиданных значений None на 0
for i in range(len(sequences_goods)):
    sequences_goods[i] = [0 if v is None else v for v in sequences_goods[i]]

x1 = pad_sequences(sequences_goods, maxlen=max_position_len)

In [10]:
del sequences_goods

In [11]:
zFullPred = pd.DataFrame(columns=['category_id'])
while len(x1) > SLICE:
    x = x1[:SLICE]
    y = model.predict(x)
    yPred = pd.DataFrame(y, columns=dummiesColumnsName)
    zPred = pd.DataFrame(pd.get_dummies(yPred).idxmax(1),
                         columns=['category_id'])
    zPred['category_id'] = zPred['category_id'].astype('uint8')
    zFullPred = zFullPred.append(zPred)
    x1 = x1[SLICE:]
x = x1
y = model.predict(x)
yPred = pd.DataFrame(y, columns=dummiesColumnsName)
zPred = pd.DataFrame(pd.get_dummies(yPred).idxmax(1), columns=['category_id'])
zPred['category_id'] = zPred['category_id'].astype('uint8')
zFullPred = zFullPred.append(zPred)

zFullPred['category_id'] = zFullPred['category_id'].astype('uint8')
zFullPred.to_csv('result.csv', index=False)

In [12]:
# Удалить все, что после
z_real = pd.DataFrame(df['category_id'].astype('uint8'), columns=['category_id'])

In [14]:
print(accuracy_score(zFullPred, z_real))
#print(f1_score(zFullPred, z_real))
print(zFullPred.head())

0.7886353182813618
   category_id
0           78
1           71
2           71
3           70
4           84
